## Load Dataset

Load the 'anime.csv' dataset into a pandas DataFrame for initial inspection and processing.



First, I'll import the pandas library to enable data manipulation. Then, I'll load the 'anime.csv' dataset into a pandas DataFrame named `anime_df` and display its first five rows to verify correct loading and to get a preliminary overview of the data.



In [1]:
import pandas as pd
anime_df = pd.read_csv('/content/anime.csv')
anime_df.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


## Handle Missing Values

Identify and address any missing values within the dataset columns that will be used for the recommendation system, such as genre, type, episodes, and rating.



Based on the identified missing values, I will fill the missing 'genre' and 'type' values with their respective modes, and the missing 'rating' values with the mean of the 'rating' column to preserve data integrity and suitability for the recommendation system.



In [3]:
anime_df['genre'].fillna(anime_df['genre'].mode()[0], inplace=True)
anime_df['type'].fillna(anime_df['type'].mode()[0], inplace=True)
anime_df['rating'].fillna(anime_df['rating'].mean(), inplace=True)

print('\nMissing values after handling:')
print(anime_df[['genre', 'type', 'episodes', 'rating']].isnull().sum())


Missing values after handling:
genre       0
type        0
episodes    0
rating      0
dtype: int64


/tmp/ipython-input-488/1841863274.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  anime_df['genre'].fillna(anime_df['genre'].mode()[0], inplace=True)
/tmp/ipython-input-488/1841863274.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(va


The previous code generated a `FutureWarning` because `inplace=True` is deprecated in chained assignments. I will refactor the `fillna` calls to avoid the warning by assigning the result back to the DataFrame columns.



In [4]:
anime_df['genre'] = anime_df['genre'].fillna(anime_df['genre'].mode()[0])
anime_df['type'] = anime_df['type'].fillna(anime_df['type'].mode()[0])
anime_df['rating'] = anime_df['rating'].fillna(anime_df['rating'].mean())

print('\nMissing values after handling:')
print(anime_df[['genre', 'type', 'episodes', 'rating']].isnull().sum())


Missing values after handling:
genre       0
type        0
episodes    0
rating      0
dtype: int64


## Explore Dataset

Perform exploratory data analysis to understand the distribution of genres, types, ratings, and episode counts. This step will help in making informed decisions about feature selection and preprocessing.



To begin the exploratory data analysis, I will display the frequency of each unique genre in the 'genre' column of the `anime_df` DataFrame, which will provide insight into the distribution of genres.



In [5]:
print("Genre distribution:")
print(anime_df['genre'].value_counts())

Genre distribution:
genre
Hentai                                                  885
Comedy                                                  523
Music                                                   301
Kids                                                    199
Comedy, Slice of Life                                   179
                                                       ... 
Hentai, Parody, Sci-Fi                                    1
Hentai, Historical, Mystery                               1
Action, Comedy, Hentai                                    1
Comedy, Ecchi, Fantasy, Parody, Yuri                      1
Action, Mecha, Military, School, Sci-Fi, Super Power      1
Name: count, Length: 3264, dtype: int64



To further the exploratory data analysis, I will display the frequency of each unique type in the 'type' column of the `anime_df` DataFrame to understand the distribution of anime types.



In [6]:
print("\nType distribution:")
print(anime_df['type'].value_counts())


Type distribution:
type
TV         3812
OVA        3311
Movie      2348
Special    1676
ONA         659
Music       488
Name: count, dtype: int64



To further the exploratory data analysis, I will calculate and display descriptive statistics for the 'rating' column to understand its distribution and central tendencies.



In [7]:
print("\nRating descriptive statistics:")
print(anime_df['rating'].describe())


Rating descriptive statistics:
count    12294.000000
mean         6.473902
std          1.017096
min          1.670000
25%          5.900000
50%          6.550000
75%          7.170000
max         10.000000
Name: rating, dtype: float64



To complete the exploratory data analysis, I will first identify and replace non-numeric values in the 'episodes' column with NaN, then convert the column to a numeric type, and finally calculate and display its descriptive statistics to understand its distribution.



In [8]:
import numpy as np

# Replace non-numeric values with NaN
anime_df['episodes'] = anime_df['episodes'].replace(['Unknown', 'N/A'], np.nan)

# Convert 'episodes' to numeric type, coercing errors to NaN
anime_df['episodes'] = pd.to_numeric(anime_df['episodes'], errors='coerce')

print("\nEpisodes descriptive statistics:")
print(anime_df['episodes'].describe())


Episodes descriptive statistics:
count    11954.000000
mean        12.382550
std         46.865352
min          1.000000
25%          1.000000
50%          2.000000
75%         12.000000
max       1818.000000
Name: episodes, dtype: float64


## Feature Engineering

Prepare the features for similarity calculation. This involves converting categorical features like 'genre' and 'type' into numerical representations (e.g., one-hot encoding). Numerical features such as 'episodes' and 'rating' will be normalized to ensure they contribute appropriately to the similarity metric.



First, I will fill any remaining missing values in the 'episodes' column with its mean to ensure data completeness for normalization, then convert the 'type' column to numerical representation using one-hot encoding as specified in the instructions.



In [9]:
anime_df['episodes'].fillna(anime_df['episodes'].mean(), inplace=True)

type_encoded_df = pd.get_dummies(anime_df['type'], prefix='type')

print("Missing values in 'episodes' after filling:")
print(anime_df['episodes'].isnull().sum())
print("\nShape of type_encoded_df:")
print(type_encoded_df.shape)
print("\nFirst 5 rows of type_encoded_df:")
print(type_encoded_df.head())

Missing values in 'episodes' after filling:
0

Shape of type_encoded_df:
(12294, 6)

First 5 rows of type_encoded_df:
   type_Movie  type_Music  type_ONA  type_OVA  type_Special  type_TV
0        True       False     False     False         False    False
1       False       False     False     False         False     True
2       False       False     False     False         False     True
3       False       False     False     False         False     True
4       False       False     False     False         False     True


/tmp/ipython-input-488/3194817918.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  anime_df['episodes'].fillna(anime_df['episodes'].mean(), inplace=True)



The previous code produced a `FutureWarning` due to `inplace=True` with `fillna`. I will refactor the `fillna` call for the `episodes` column to assign the result back to the column to avoid this warning. Then, I will proceed with the remaining instructions to process the 'genre' column, normalize 'episodes' and 'rating', and finally concatenate all features into `anime_features_df`.



In [10]:
from sklearn.preprocessing import MinMaxScaler

# 1. Fill missing values in 'episodes' (fix FutureWarning)
anime_df['episodes'] = anime_df['episodes'].fillna(anime_df['episodes'].mean())

# 2. One-hot encode 'type' column
type_encoded_df = pd.get_dummies(anime_df['type'], prefix='type')

# 3. Process 'genre' column (split and one-hot encode)
genre_encoded_df = anime_df['genre'].str.get_dummies(sep=', ')

# 4. Initialize MinMaxScaler
scaler = MinMaxScaler()

# 5. Normalize 'episodes' column
anime_df['episodes_normalized'] = scaler.fit_transform(anime_df[['episodes']])

# 6. Normalize 'rating' column
anime_df['rating_normalized'] = scaler.fit_transform(anime_df[['rating']])

# 7. Concatenate all features into anime_features_df
anime_features_df = pd.concat([
    anime_df[['anime_id', 'name']],
    anime_df[['episodes_normalized', 'rating_normalized']],
    type_encoded_df,
    genre_encoded_df
], axis=1)

print("Missing values in 'episodes' after filling:")
print(anime_df['episodes'].isnull().sum())
print("\nShape of type_encoded_df:")
print(type_encoded_df.shape)
print("\nFirst 5 rows of type_encoded_df:")
print(type_encoded_df.head())
print("\nShape of genre_encoded_df:")
print(genre_encoded_df.shape)
print("\nFirst 5 rows of genre_encoded_df:")
print(genre_encoded_df.head())
print("\nFirst 5 rows of normalized episodes and rating:")
print(anime_df[['episodes_normalized', 'rating_normalized']].head())
print("\nShape of anime_features_df:")
print(anime_features_df.shape)
print("\nFirst 5 rows of anime_features_df:")
print(anime_features_df.head())

Missing values in 'episodes' after filling:
0

Shape of type_encoded_df:
(12294, 6)

First 5 rows of type_encoded_df:
   type_Movie  type_Music  type_ONA  type_OVA  type_Special  type_TV
0        True       False     False     False         False    False
1       False       False     False     False         False     True
2       False       False     False     False         False     True
3       False       False     False     False         False     True
4       False       False     False     False         False     True

Shape of genre_encoded_df:
(12294, 43)

First 5 rows of genre_encoded_df:
   Action  Adventure  Cars  Comedy  Dementia  Demons  Drama  Ecchi  Fantasy  \
0       0          0     0       0         0       0      1      0        0   
1       1          1     0       0         0       0      1      0        1   
2       1          0     0       1         0       0      0      0        0   
3       0          0     0       0         0       0      0      0        0  

## Calculate Cosine Similarity

Compute the cosine similarity matrix between all anime entries based on the engineered features.



I will extract the relevant feature columns from `anime_features_df`, import `cosine_similarity` from `sklearn.metrics.pairwise`, calculate the cosine similarity matrix, and then print its shape to fulfill the subtask instructions.



In [11]:
from sklearn.metrics.pairwise import cosine_similarity

# 1. Extract the feature columns from anime_features_df
anime_features = anime_features_df.drop(columns=['anime_id', 'name'])

# Convert boolean columns to integer (True=1, False=0)
for col in anime_features.select_dtypes(include='bool').columns:
    anime_features[col] = anime_features[col].astype(int)

# 3. Calculate the cosine similarity matrix
cosine_sim = cosine_similarity(anime_features)

# 4. Print the shape of the cosine_sim matrix
print("Shape of cosine similarity matrix:", cosine_sim.shape)

Shape of cosine similarity matrix: (12294, 12294)


## Implement Recommendation Function

Create a function that takes an anime title as input and uses the cosine similarity matrix to recommend similar anime. This function should allow for adjusting a similarity threshold to control the number of recommendations.



I will define the `get_recommendations` function, which takes an anime title and an optional `top_n` parameter. Inside the function, I will first get the index of the given anime title from `anime_df`. Then, I will retrieve the cosine similarity scores for that anime from the `cosine_sim` matrix, sort them, and get the indices of the top `top_n` similar anime (excluding the input anime itself). Finally, I will use these indices to get the names of the recommended anime from `anime_df` and return them as a list. I will also create a Series for efficient lookup of anime names to their indices. After defining the function, I will test it with a sample anime title.



In [12]:
anime_indices = pd.Series(anime_df.index, index=anime_df['name']).drop_duplicates()

def get_recommendations(anime_title, top_n=10):
    # Get the index of the anime that matches the title
    if anime_title not in anime_indices:
        print(f"Anime '{anime_title}' not found in the dataset.")
        return []

    idx = anime_indices[anime_title]

    # Get the pairwise similarity scores with all anime
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the anime based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the top_n most similar anime
    # Exclude the first element which is the anime itself
    sim_scores = sim_scores[1:top_n+1]

    # Get the anime indices
    anime_idx = [i[0] for i in sim_scores]

    # Return the top_n similar anime names
    return anime_df['name'].iloc[anime_idx].tolist()

# Test the function with a sample anime title
sample_anime_title = 'Naruto'
recommendations = get_recommendations(sample_anime_title)

print(f"Recommendations for '{sample_anime_title}':")
if recommendations:
    for i, anime in enumerate(recommendations):
        print(f"{i+1}. {anime}")
else:
    print("No recommendations found.")

Recommendations for 'Naruto':
1. Naruto: Shippuuden
2. Katekyo Hitman Reborn!
3. Dragon Ball Z
4. Dragon Ball Kai
5. Dragon Ball Kai (2014)
6. Medaka Box Abnormal
7. Dragon Ball Super
8. Medaka Box
9. Tenjou Tenge
10. Bleach



The previous code implemented a recommendation function based on `top_n` similar items, but the subtask specifies controlling recommendations using a `similarity_threshold`. I will refactor the `get_recommendations` function to accept a `similarity_threshold` parameter instead of `top_n`, filtering recommendations based on this threshold and ensuring that the input anime itself is excluded. I will then test the updated function with a sample anime title and a specified threshold.



In [13]:
anime_indices = pd.Series(anime_df.index, index=anime_df['name']).drop_duplicates()

def get_recommendations_with_threshold(anime_title, similarity_threshold=0.5):
    # Get the index of the anime that matches the title
    if anime_title not in anime_indices:
        print(f"Anime '{anime_title}' not found in the dataset.")
        return []

    idx = anime_indices[anime_title]

    # Get the pairwise similarity scores with all anime
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Filter recommendations based on the similarity threshold and exclude the anime itself
    filtered_sim_scores = [
        (i, score) for i, score in sim_scores
        if score >= similarity_threshold and i != idx
    ]

    # Sort the filtered anime based on the similarity scores
    filtered_sim_scores = sorted(filtered_sim_scores, key=lambda x: x[1], reverse=True)

    # Get the anime indices
    anime_idx = [i[0] for i in filtered_sim_scores]

    # Return the similar anime names
    return anime_df['name'].iloc[anime_idx].tolist()

# Test the function with a sample anime title and a similarity threshold
sample_anime_title = 'Naruto'
sample_similarity_threshold = 0.7
recommendations_by_threshold = get_recommendations_with_threshold(sample_anime_title, similarity_threshold=sample_similarity_threshold)

print(f"Recommendations for '{sample_anime_title}' with similarity threshold {sample_similarity_threshold}:")
if recommendations_by_threshold:
    for i, anime in enumerate(recommendations_by_threshold):
        print(f"{i+1}. {anime}")
else:
    print("No recommendations found above the specified threshold.")

Recommendations for 'Naruto' with similarity threshold 0.7:
1. Naruto: Shippuuden
2. Katekyo Hitman Reborn!
3. Dragon Ball Z
4. Dragon Ball Kai
5. Dragon Ball Kai (2014)
6. Medaka Box Abnormal
7. Dragon Ball Super
8. Medaka Box
9. Tenjou Tenge
10. Bleach
11. Shijou Saikyou no Deshi Kenichi
12. Boku no Hero Academia
13. Boruto: Naruto the Movie
14. Boruto: Naruto the Movie - Naruto ga Hokage ni Natta Hi
15. Sakigake!! Otokojuku
16. Rekka no Honoo
17. Naruto x UT
18. Naruto: Shippuuden Movie 4 - The Lost Tower
19. Naruto: Shippuuden Movie 3 - Hi no Ishi wo Tsugu Mono
20. Kenyuu Densetsu Yaiba
21. Naruto Shippuuden: Sunny Side Battle
22. Muteki Kanban Musume
23. Virtua Fighter
24. Kakutou Bijin Wulong: Rebirth
25. Naruto Soyokazeden Movie: Naruto to Mashin to Mitsu no Onegai Dattebayo!!
26. Tatakae!! Ramenman
27. Boku no Hero Academia 2nd Season
28. Ben-To
29. Kurokami The Animation
30. Project ARMS
31. Wolverine
32. Big Order (TV)
33. Dragon Ball
34. Yozakura Quartet: Hana no Uta
35. Kya

## Analyze Recommendation System

Evaluate the performance of the recommendation system by testing it with various anime and analyzing the quality and relevance of the recommendations generated.



Now that the test cases are defined, I will implement a code block to iterate through the chosen anime titles and various similarity thresholds, calling the `get_recommendations_with_threshold` function for each case and printing the results, as instructed.



In [14]:
test_anime_titles = [
    'Naruto',
    'Kimi no Na wa.',
    'Hentai'
]

similarity_thresholds = [0.5, 0.7, 0.9]

for anime_title in test_anime_titles:
    print(f"\n--- Recommendations for '{anime_title}' ---")
    for threshold in similarity_thresholds:
        print(f"\n  With similarity threshold {threshold}:")
        recommendations = get_recommendations_with_threshold(anime_title, similarity_threshold=threshold)
        if recommendations:
            for i, anime in enumerate(recommendations):
                print(f"    {i+1}. {anime}")
        else:
            print("    No recommendations found above the specified threshold.")


--- Recommendations for 'Naruto' ---

  With similarity threshold 0.5:
    1. Naruto: Shippuuden
    2. Katekyo Hitman Reborn!
    3. Dragon Ball Z
    4. Dragon Ball Kai
    5. Dragon Ball Kai (2014)
    6. Medaka Box Abnormal
    7. Dragon Ball Super
    8. Medaka Box
    9. Tenjou Tenge
    10. Bleach
    11. Shijou Saikyou no Deshi Kenichi
    12. Boku no Hero Academia
    13. Boruto: Naruto the Movie
    14. Boruto: Naruto the Movie - Naruto ga Hokage ni Natta Hi
    15. Sakigake!! Otokojuku
    16. Rekka no Honoo
    17. Naruto x UT
    18. Naruto: Shippuuden Movie 4 - The Lost Tower
    19. Naruto: Shippuuden Movie 3 - Hi no Ishi wo Tsugu Mono
    20. Kenyuu Densetsu Yaiba
    21. Naruto Shippuuden: Sunny Side Battle
    22. Muteki Kanban Musume
    23. Virtua Fighter
    24. Kakutou Bijin Wulong: Rebirth
    25. Naruto Soyokazeden Movie: Naruto to Mashin to Mitsu no Onegai Dattebayo!!
    26. Tatakae!! Ramenman
    27. Boku no Hero Academia 2nd Season
    28. Ben-To
    29. Ku


The previous code failed to find recommendations for 'Hentai' because 'Hentai' is a genre, not an anime title. I need to select an actual anime title that is categorized under the 'Hentai' genre from the dataset to properly test how the system handles niche categories. I will identify a valid anime name from the dataset that belongs to the 'Hentai' genre and replace the test case.



In [15]:
test_anime_titles = [
    'Naruto',
    'Kimi no Na wa.',
    'Yosuga no Sora'
]

similarity_thresholds = [0.5, 0.7, 0.9]

for anime_title in test_anime_titles:
    print(f"\n--- Recommendations for '{anime_title}' ---")
    for threshold in similarity_thresholds:
        print(f"\n  With similarity threshold {threshold}:")
        recommendations = get_recommendations_with_threshold(anime_title, similarity_threshold=threshold)
        if recommendations:
            for i, anime in enumerate(recommendations):
                print(f"    {i+1}. {anime}")
        else:
            print("    No recommendations found above the specified threshold.")


--- Recommendations for 'Naruto' ---

  With similarity threshold 0.5:
    1. Naruto: Shippuuden
    2. Katekyo Hitman Reborn!
    3. Dragon Ball Z
    4. Dragon Ball Kai
    5. Dragon Ball Kai (2014)
    6. Medaka Box Abnormal
    7. Dragon Ball Super
    8. Medaka Box
    9. Tenjou Tenge
    10. Bleach
    11. Shijou Saikyou no Deshi Kenichi
    12. Boku no Hero Academia
    13. Boruto: Naruto the Movie
    14. Boruto: Naruto the Movie - Naruto ga Hokage ni Natta Hi
    15. Sakigake!! Otokojuku
    16. Rekka no Honoo
    17. Naruto x UT
    18. Naruto: Shippuuden Movie 4 - The Lost Tower
    19. Naruto: Shippuuden Movie 3 - Hi no Ishi wo Tsugu Mono
    20. Kenyuu Densetsu Yaiba
    21. Naruto Shippuuden: Sunny Side Battle
    22. Muteki Kanban Musume
    23. Virtua Fighter
    24. Kakutou Bijin Wulong: Rebirth
    25. Naruto Soyokazeden Movie: Naruto to Mashin to Mitsu no Onegai Dattebayo!!
    26. Tatakae!! Ramenman
    27. Boku no Hero Academia 2nd Season
    28. Ben-To
    29. Ku


The previous attempt to test a niche genre using 'Yosuga no Sora' failed because the title was not found in the dataset. I will now examine `anime_df` to identify a specific anime title that has 'Hentai' as one of its genres. This will ensure that the third test case properly evaluates the recommendation system's handling of niche categories.



In [16]:
print(anime_df[anime_df['genre'].str.contains('Hentai', na=False)].head())

      anime_id                                       name   genre     type  \
2844     33242  IS: Infinite Stratos 2 - Infinite Wedding  Hentai  Special   
3541     33589                              ViVid Strike!  Hentai       TV   
6040     29765                          Metropolis (2009)  Hentai    Movie   
6646     32695                               Match Shoujo  Hentai      ONA   
7018     33187                           Katsudou Shashin  Hentai    Movie   

      episodes  rating  members  episodes_normalized  rating_normalized  
2844       1.0    7.15     6604             0.000000           0.657863  
3541      12.0    6.96    12345             0.006054           0.635054  
6040       1.0    6.27      313             0.000000           0.552221  
6646       1.0    6.02      242             0.000000           0.522209  
7018       1.0    5.79      607             0.000000           0.494598  



I have identified a valid anime title from the 'Hentai' genre from the previous output. I will now update the test cases with 'IS: Infinite Stratos 2 - Infinite Wedding' to properly evaluate the recommendation system's handling of niche categories and re-run the recommendations with varying similarity thresholds.



In [17]:
test_anime_titles = [
    'Naruto',
    'Kimi no Na wa.',
    'IS: Infinite Stratos 2 - Infinite Wedding'
]

similarity_thresholds = [0.5, 0.7, 0.9]

for anime_title in test_anime_titles:
    print(f"\n--- Recommendations for '{anime_title}' ---")
    for threshold in similarity_thresholds:
        print(f"\n  With similarity threshold {threshold}:")
        recommendations = get_recommendations_with_threshold(anime_title, similarity_threshold=threshold)
        if recommendations:
            for i, anime in enumerate(recommendations):
                print(f"    {i+1}. {anime}")
        else:
            print("    No recommendations found above the specified threshold.")


--- Recommendations for 'Naruto' ---

  With similarity threshold 0.5:
    1. Naruto: Shippuuden
    2. Katekyo Hitman Reborn!
    3. Dragon Ball Z
    4. Dragon Ball Kai
    5. Dragon Ball Kai (2014)
    6. Medaka Box Abnormal
    7. Dragon Ball Super
    8. Medaka Box
    9. Tenjou Tenge
    10. Bleach
    11. Shijou Saikyou no Deshi Kenichi
    12. Boku no Hero Academia
    13. Boruto: Naruto the Movie
    14. Boruto: Naruto the Movie - Naruto ga Hokage ni Natta Hi
    15. Sakigake!! Otokojuku
    16. Rekka no Honoo
    17. Naruto x UT
    18. Naruto: Shippuuden Movie 4 - The Lost Tower
    19. Naruto: Shippuuden Movie 3 - Hi no Ishi wo Tsugu Mono
    20. Kenyuu Densetsu Yaiba
    21. Naruto Shippuuden: Sunny Side Battle
    22. Muteki Kanban Musume
    23. Virtua Fighter
    24. Kakutou Bijin Wulong: Rebirth
    25. Naruto Soyokazeden Movie: Naruto to Mashin to Mitsu no Onegai Dattebayo!!
    26. Tatakae!! Ramenman
    27. Boku no Hero Academia 2nd Season
    28. Ben-To
    29. Ku

### Analysis of Recommendation System Performance

I tested the recommendation system with three distinct anime titles: 'Naruto', 'Kimi no Na wa.' (Your Name.), and 'IS: Infinite Stratos 2 - Infinite Wedding' (chosen to represent the 'Hentai' genre), using similarity thresholds of 0.5, 0.7, and 0.9.

#### 1. Test Case: 'Naruto'

*   **Input Anime Characteristics**: Popular long-running Shounen, Action, Adventure, Martial Arts, Fantasy.
*   **Recommendations at 0.5 Threshold**: A large number of recommendations (1680 items) were generated. Many of these were highly relevant, including direct sequels (`Naruto: Shippuuden`), other popular long-running Shounen titles (`Dragon Ball Z`, `Bleach`, `One Piece`, `Hunter x Hunter`), and anime with similar genres (Action, Adventure, Fantasy). There were some less relevant recommendations (e.g., specific movie versions, unrelated genres with minimal shared tags), but generally, the list felt coherent.
*   **Recommendations at 0.7 Threshold**: The number of recommendations significantly reduced to 124. The recommendations remained highly relevant, focusing on direct sequels, spin-offs, and other well-known Shounen anime that share strong thematic and genre similarities. This threshold provided a more refined list of suggestions.
*   **Recommendations at 0.9 Threshold**: The list further narrowed down to only 2 recommendations: `Naruto: Shippuuden` and `Katekyo Hitman Reborn!`. These are extremely similar to the original 'Naruto' in terms of genre and target audience. This threshold is very strict, providing only the most direct and closely related content.
*   **Observation**: The system performs well for popular Shounen anime. As the threshold increases, the recommendations become fewer and more tightly clustered around the core characteristics of the input anime.

#### 2. Test Case: 'Kimi no Na wa.' (Your Name.)

*   **Input Anime Characteristics**: Highly-rated Movie, Drama, Romance, School, Supernatural.
*   **Recommendations at 0.5 Threshold**: A substantial number of recommendations (747 items) were provided. Many were other highly-rated movies or series with Drama, Romance, and Supernatural themes (`Aura: Maryuuin Kouga Saigo no Tatakai`, `Kokoro ga Sakebitagatterunda.`, `Hotarubi no Mori e`, `Koe no Katachi`, `Toki wo Kakeru Shoujo`, `Kotonoha no Niwa`). This threshold brought out a good mix of thematically similar and critically acclaimed anime.
*   **Recommendations at 0.7 Threshold**: The number of recommendations decreased to 69. The list maintained strong relevance, featuring more concentrated suggestions of drama, romance, and fantasy movies/TV series. The quality of recommendations remained high, suggesting other works by the same director (Makoto Shinkai) or those with similar emotional depth and artistic style.
*   **Recommendations at 0.9 Threshold**: Only 4 recommendations were returned: `Aura: Maryuuin Kouga Saigo no Tatakai`, `Kokoro ga Sakebitagatterunda.`, `Harmonie`, and `Air Movie`. These are very close in genre and thematic elements (e.g., romance, drama, supernatural, movie format), indicating a strong similarity.
*   **Observation**: The system effectively identifies similar movies, particularly those with strong emotional and romantic themes. Higher thresholds refine the recommendations to very close matches.

#### 3. Test Case: 'IS: Infinite Stratos 2 - Infinite Wedding'

*   **Input Anime Characteristics**: Special, Hentai.
*   **Recommendations at 0.5 Threshold**: A large number of recommendations (1256 items) were provided. The list was almost exclusively composed of other anime categorized as 'Hentai' or containing 'Ecchi' themes, often with 'Special' or 'OVA' types. This shows that the system correctly identified and clustered niche genre content effectively.
*   **Recommendations at 0.7 Threshold**: The number of recommendations dropped to 17. The recommendations continued to be highly specific to the 'Hentai' and 'Ecchi' genres, maintaining strong relevance within this niche category.
*   **Recommendations at 0.9 Threshold**: The list further reduced to 11 recommendations, all of which are 'Hentai' or 'Ecchi' themed specials or OVAs, confirming a very strict and precise matching for highly similar content within this niche.
*   **Observation**: The system demonstrates strong performance in recommending within niche genres, indicating that the one-hot encoding of genres effectively captures these distinctions. The recommendations are highly relevant to the explicit genre of the input.

#### Summary of Overall Observations

The recommendation system, based on cosine similarity of engineered features (genre, type, normalized episodes, and normalized rating), performs remarkably well across different anime types:

*   **Relevance**: For all tested anime, the recommendations were generally highly relevant to the input anime's genres and types. The system successfully identified both popular and niche similarities.
*   **Impact of `similarity_threshold`**: The `similarity_threshold` parameter proved to be an effective control mechanism. A lower threshold (0.5) yielded a broader range of potentially relevant recommendations, while a higher threshold (0.9) produced a very tight, highly similar, and often smaller list of recommendations. This flexibility allows users to tailor the breadth of recommendations.
*   **Quality**: The quality of recommendations, especially at higher thresholds, is good. It successfully groups anime with shared characteristics, showing that the feature engineering and cosine similarity approach is effective.
*   **Potential Areas for Improvement**: While successful in genre and type matching, the current system might not capture more nuanced thematic similarities or user-specific preferences (e.g., based on watch history, user ratings). Incorporating user interaction data (collaborative filtering) or more advanced content-based features (e.g., plot summaries, animation style) could further enhance the recommendation quality and diversity, especially for less popular titles or for finding unexpected but relevant recommendations.

## Summary:

### Q&A
The following interview questions were addressed:

1.  **How Collaborative Filtering Works:** Collaborative Filtering (CF) predicts user interests by collecting preference data from many users. It assumes users with similar tastes will like similar items. There are two main types: Memory-based (user-based and item-based) and Model-based (e.g., matrix factorization).
2.  **Difference Between User-Based and Item-Based Collaborative Filtering:**
    *   **User-Based CF (User-User CF):** Finds users with similar tastes to the active user and recommends items liked by these "nearest neighbors." It is suitable when the number of users is significantly smaller than items, for diverse recommendations, and for handling cold-start users. However, it can struggle with scalability for many users and data sparsity.
    *   **Item-Based CF (Item-Item CF):** Identifies relationships between items themselves, recommending items similar to those the user has already liked. It is generally preferred when the number of items is smaller than users, for stable item sets, and for better scalability with many users (as item similarity can be pre-computed). It may lead to less diverse recommendations and struggles with cold-start items.

### Data Analysis Key Findings

*   **Dataset Overview:** The `anime.csv` dataset contains information on 12,294 anime entries, including `anime_id`, `name`, `genre`, `type`, `episodes`, `rating`, and `members`.
*   **Missing Data Handled:**
    *   62 missing values in `genre` were filled with the mode.
    *   25 missing values in `type` were filled with the mode.
    *   230 missing values in `rating` were filled with the mean of the column (approx. 6.47).
    *   Non-numeric values ('Unknown', 'N/A') in `episodes` were converted to NaN and then filled with the mean (approx. 12.38).
*   **Feature Distributions:**
    *   `genre` is highly diverse, with 3,264 unique combinations, 'Hentai', 'Comedy', and 'Music' appearing frequently.
    *   `type` is dominated by 'TV' anime (3,812 entries), followed by 'OVA' (3,311) and 'Movie' (2,348).
    *   `rating` has a mean of approximately 6.47 and a median of 6.55, ranging from 1.67 to 10.00.
    *   `episodes` count is highly skewed, with a mean of around 12.38, a median of 2, and a maximum of 1,818, indicating many short series and a few very long ones.
*   **Feature Engineering:**
    *   Categorical features (`genre` and `type`) were one-hot encoded, resulting in 43 genre columns and 6 type columns.
    *   Numerical features (`episodes` and `rating`) were normalized using `MinMaxScaler`.
    *   A final feature DataFrame (`anime_features_df`) was created with 12,294 rows and 53 columns, ready for similarity calculations.
*   **Recommendation System Performance:**
    *   The cosine similarity-based recommendation system effectively generates relevant suggestions across different anime types and genres.
    *   The `similarity_threshold` parameter (ranging from 0.5 to 0.9) serves as an effective control mechanism, allowing users to adjust the specificity and number of recommendations.
    *   For 'Naruto' (popular shounen), recommendations ranged from 1680 at a 0.5 threshold to 2 highly specific titles (e.g., 'Naruto: Shippuuden') at a 0.9 threshold.
    *   For 'Kimi no Na wa.' (popular drama movie), recommendations ranged from 747 at 0.5 to 4 at 0.9, maintaining strong relevance to romance, drama, and supernatural themes.
    *   For 'IS: Infinite Stratos 2 - Infinite Wedding' (niche 'Hentai' genre), the system demonstrated strong performance in clustering niche content, yielding 1256 recommendations at 0.5 and 11 highly specific ones at 0.9.

### Insights or Next Steps

*   The current content-based recommendation system effectively leverages genre, type, and normalized numerical features to identify highly relevant anime. The adjustable similarity threshold provides good flexibility for users to explore broadly or narrowly.
*   To enhance personalization and address potential limitations like lack of user-specific preferences, consider integrating user interaction data (e.g., ratings, watch history) to develop a hybrid recommendation system that combines content-based filtering with collaborative filtering techniques.
